In [21]:
!pip install folium
!pip install ipywidgets
!pip install geocoder
!pip install geopy
!pip install pandas
!pip install openeo
!pip install xarray
!pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.6/265.6 kB 4.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.9/182.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 9.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.2/323.2 kB 7.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 19.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 13.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 13.4 MB/s eta 0:00:00


## Imports

In [1]:
import folium
from folium import plugins
import ipywidgets
import geocoder
import geopy
import numpy as np
import pandas as pd
import json
#from vega_datasets import data as vds
#hola

## Mapa interactiu

In [ ]:
!pip install folium ipyleaflet

In [15]:
import folium
from ipyleaflet import Map, DrawControl
import json

# Crear un mapa centrado en una ubicación específica
m = Map(center=(41.3851, 2.1734), zoom=12)  # Coordenadas de Barcelona

# Añadir control de dibujo
draw_control = DrawControl()
m.add_control(draw_control)

# Función para guardar los datos en un archivo JSON
def save_geojson(data, filename='selected_area.json'):
    with open(filename, 'w') as f:
        json.dump(data, f)

# Evento para capturar la selección del área
def handle_draw(target, action, geo_json):
    if action == 'created':
        save_geojson(geo_json)
        print(f"Área guardada en {filename}")

draw_control.on_draw(handle_draw)

# Mostrar el mapa
m


Map(center=[41.3851, 2.1734], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

In [16]:
# Función para cargar los datos desde un archivo JSON
def load_geojson(filename='selected_area.json'):
    with open(filename, 'r') as f:
        data = json.load(f)
    return data

# Cargar los datos
selected_area = load_geojson()
print(selected_area)

{'type': 'Feature', 'properties': {'style': {'stroke': True, 'color': '#3388ff', 'weight': 4, 'opacity': 0.5, 'fill': True, 'fillColor': None, 'fillOpacity': 0.2, 'clickable': True}}, 'geometry': {'type': 'Polygon', 'coordinates': [[[2.05307, 41.417768], [2.052727, 41.372954], [2.111778, 41.368574], [2.106285, 41.407726], [2.072983, 41.426263], [2.05307, 41.417768]]]}}


## Importar dades

In [18]:
import openeo # https://openeo.org/documentation/1.0/python/#collections
import xarray as xr
import matplotlib.pyplot as plt

# Connexió al "Backend" d'on extreurèm les dades
backend = "openeo.dataspace.copernicus.eu"
conn = openeo.connect(backend).authenticate_oidc()

Authenticated using refresh token.


In [19]:
# LLista de tots els satèlits disponibles
conn.list_collections()

[{'description': 'Sentinel 3 imagery captured by OLCI sensor',
  'extent': {'spatial': {'bbox': [[-180.0, -85.0, 180.0, 85.0]]},
   'temporal': {'interval': [['2016-04-17T11:33:13Z', None]]}},
  'id': 'SENTINEL3_OLCI_L1B',
  'license': 'proprietary',
  'links': [{'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/',
    'rel': 'root',
    'type': 'application/json'},
   {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-3-olci',
    'rel': 'self',
    'type': 'application/json'},
   {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections',
    'rel': 'parent',
    'type': 'application/json'},
   {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-3-olci/queryables',
    'rel': 'http://www.opengis.net/def/rel/ogc/1.0/queryables',
    'type': 'application/schema+json'},
   {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-3-olci/items',
    'rel': 'items',
    'type': 'application/geo+json'}],
  'providers': [],
  'stac_extensions': ['https://stac-extensions.github.io/scientific/v1.0.0/schema.json',
   'https://stac-extensions.github.io/sat/v1.0.0/schema.json',
   'https://stac-extensions.github.io/eo/v1.0.0/schema.json'],
  'stac_version': '1.0.0',
  'title': 'Sentinel 3 OLCI'},
 {'description': 'Sentinel 3 imagery captured by SLSTR sensor',
  'extent': {'spatial': {'bbox': [[-180.0, -85.0, 180.0, 85.0]]},
   'temporal': {'interval': [['2016-04-17T11:33:13Z', None]]}},
  'id': 'SENTINEL3_SLSTR',
  'license': 'proprietary',
  'links': [{'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/',
    'rel': 'root',
    'type': 'application/json'},
   {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-3-slstr',
    'rel': 'self',
    'type': 'application/json'},
   {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections',
    'rel': 'parent',
    'type': 'application/json'},
   {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-3-slstr/queryables',
    'rel': 'http://www.opengis.net/def/rel/ogc/1.0/queryables',
    'type': 'application/schema+json'},
   {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-3-slstr/items',
    'rel': 'items',
    'type': 'application/geo+json'}],
  'providers': [],
  'stac_extensions': ['https://stac-extensions.github.io/scientific/v1.0.0/schema.json',
   'https://stac-extensions.github.io/sat/v1.0.0/schema.json',
   'https://stac-extensions.github.io/eo/v1.0.0/schema.json'],
  'stac_version': '1.0.0',
  'title': 'Sentinel 3 SLSTR'},
 {'description': 'Sentinel 5 Precursor imagery captured by TROPOMI sensor.\n\nThis dataset only supports loading one band at a time.',
  'extent': {'spatial': {'bbox': [[-180.0, -85.0, 180.0, 85.0]]},
   'temporal': {'interval': [['2018-04-30T00:18:50Z', None]]}},
  'id': 'SENTINEL_5P_L2',
  'license': 'proprietary',
  'links': [{'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/',
    'rel': 'root',
    'type': 'application/json'},
   {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-5p-l2',
    'rel': 'self',
    'type': 'application/json'},
   {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections',
    'rel': 'parent',
    'type': 'application/json'},
   {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-5p-l2/queryables',
    'rel': 'http://www.opengis.net/def/rel/ogc/1.0/queryables',
    'type': 'application/schema+json'},
   {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-5p-l2/items',
    'rel': 'items',
    'type': 'application/geo+json'}],
  'providers': [],
  'stac_extensions': ['https://stac-extensions.github.io/scientific/v1.0.0/schema.json',
   'https://stac-extensions.github.io/sat/v1.0.0/schema.json',
   'https://docs.sentinel-hub.c

In [27]:
# Get information on the type of data on a collection (by id)
conn.describe_collection("LANDSAT8_L2")

{'cube:dimensions': {'bands': {'type': 'bands',
   'values': ['B01',
    'B02',
    'B03',
    'B04',
    'B05',
    'B06',
    'B07',
    'B10',
    'BQA',
    'QA_RADSAT',
    'SR_QA_AEROSOL',
    'ST_QA',
    'ST_TRAD',
    'ST_URAD',
    'ST_DRAD',
    'ST_ATRAN',
    'ST_EMIS',
    'ST_EMSD',
    'ST_CDIST']},
  't': {'extent': ['2015-01-01T00:00:00Z', None], 'type': 'temporal'},
  'x': {'axis': 'x',
   'extent': [-20, 35],
   'reference_system': {'$schema': 'https://proj.org/schemas/v0.2/projjson.schema.json',
    'area': 'World',
    'bbox': {'east_longitude': 180,
     'north_latitude': 90,
     'south_latitude': -90,
     'west_longitude': -180},
    'coordinate_system': {'axis': [{'abbreviation': 'Lat',
       'direction': 'north',
       'name': 'Geodetic latitude',
       'unit': 'degree'},
      {'abbreviation': 'Lon',
       'direction': 'east',
       'name': 'Geodetic longitude',
       'unit': 'degree'}],
     'subtype': 'ellipsoidal'},
    'datum': {'ellipsoid': {'inverse_flattening': 298.257223563,
      'name': 'WGS 84',
      'semi_major_axis': 6378137},
     'name': 'World Geodetic System 1984',
     'type': 'GeodeticReferenceFrame'},
    'id': {'authority': 'OGC', 'code': 'Auto42001', 'version': '1.3'},
    'name': 'AUTO 42001 (Universal Transverse Mercator)',
    'type': 'GeodeticCRS'},
   'step': 30,
   'type': 'spatial'},
  'y': {'axis': 'y',
   'extent': [-35, 75],
   'reference_system': {'$schema': 'https://proj.org/schemas/v0.2/projjson.schema.json',
    'area': 'World',
    'bbox': {'east_longitude': 180,
     'north_latitude': 90,
     'south_latitude': -90,
     'west_longitude': -180},
    'coordinate_system': {'axis': [{'abbreviation': 'Lat',
       'direction': 'north',
       'name': 'Geodetic latitude',
       'unit': 'degree'},
      {'abbreviation': 'Lon',
       'direction': 'east',
       'name': 'Geodetic longitude',
       'unit': 'degree'}],
     'subtype': 'ellipsoidal'},
    'datum': {'ellipsoid': {'inverse_flattening': 298.257223563,
      'name': 'WGS 84',
      'semi_major_axis': 6378137},
     'name': 'World Geodetic System 1984',
     'type': 'GeodeticReferenceFrame'},
    'id': {'authority': 'OGC', 'code': 'Auto42001', 'version': '1.3'},
    'name': 'AUTO 42001 (Universal Transverse Mercator)',
    'type': 'GeodeticCRS'},
   'step': 30,
   'type': 'spatial'}},
 'description': 'Landsat-8 OLI/TIRS L2SP refers to the Level-2 Surface Reflectance product acquired by the Operational Land Imager (OLI) and Thermal Infrared Sensor (TIRS) instrument on board Landsat 8 satellite. This product is derived from the Landsat-8 OLI/TIRS L1T or L1TP product and is generated using atmospheric correction algorithms to remove atmospheric effects and convert the top-of-atmosphere reflectance to surface reflectance values.\n\n This is Landsat 8 Collection 2 European Coverage, so does not correspond to the full Landsat archive.',
 'extent': {'spatial': {'bbox': [[-20, -35, 35, 75]]},
  'temporal': {'interval': [['2015-01-01T00:00:00Z', None]]}},
 'id': 'LANDSAT8_L2',
 'license': 'proprietary',
 'links': [{'href': 'https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/atoms/files/Landsat_Data_Policy.pdf',
   'rel': 'license',
   'type': 'application/pdf'},
  {'href': 'https://catalogue.dataspace.copernicus.eu/resto/api/collections/Landsat8/search.json?productType=L2SP',
   'rel': 'opensearch',
   'title': 'CDSE OpenSearch product query link'},
  {'href': 'https://documentation.dataspace.copernicus.eu/Data/Landsat8.html#landsat-8-olitirs_l2sp',
   'rel': 'alternate',
   'title': 'CDSE documentation page',
   'type': 'application/html'},
  {'href': 'https://earth.esa.int/eogateway/catalog/landsat-8-collection-2-european-coverage',
   'rel': 'about',
   'title': 'User guide',
   'type': 'application/html'},
  {'href': 'https://openeo.dataspace.copernicus.eu/openeo/1.2/collections',
   'rel': 'root'},
  {'href': 'https://openeo.dataspace.copernicus.eu/openeo/1.2/collections',

### ↓ Filtratge de dades ↓

In [30]:
# Determinem area de busqueda (hauriem de fer que fos automàtic amb el mapa)

def read_json(filename):
    with open(filename) as input:
        field = json.load(input)
    return field

spatial_extent= read_json("selected_area.json") 



#spatial_extent = {
#    "west": 1.936725,
#    "east": 2.259583,
#    "south": 41.334633,
#    "north": 41.427328,
#}

temporal_extent = ["2024-06-01", "2024-09-14"] # Les dates estàn en notació: aaaa-mm-dd 


# Creem una variable filtre (aka: datacube)
datacube = conn.load_collection(
    "LANDSAT8_L2",
    spatial_extent=spatial_extent,
    temporal_extent= temporal_extent,
    bands=["ST_QA"], # Els noms de les bandes cambia segons satèlit
)

# Podem fer subfiltres del filtre de la forma
#avui = datacube.filter_temporal("2024-09-10", "2024-09-14")


datacube = datacube.mean_time()

result = datacube.save_result("PNG") # Diferents formats: {'CovJSON', 'CSV', 'JSON', 'PNG', 'netCDF', 'GTiff', 'Parquet'}


# Creating a new job at the back-end by sending the datacube information.
job = result.create_job()

# Starts the job and waits until it finished to download the result.
job.start_and_wait()
job.get_results().download_files("output")

# Aquesta execució pot trigar 5 min

0:00:00 Job 'j-2409140c685a4118ac14e40bd9f65b11': send 'start'
0:00:13 Job 'j-2409140c685a4118ac14e40bd9f65b11': created (progress 0%)
0:00:18 Job 'j-2409140c685a4118ac14e40bd9f65b11': created (progress 0%)
0:00:25 Job 'j-2409140c685a4118ac14e40bd9f65b11': created (progress 0%)
0:00:33 Job 'j-2409140c685a4118ac14e40bd9f65b11': created (progress 0%)
0:00:42 Job 'j-2409140c685a4118ac14e40bd9f65b11': created (progress 0%)
0:00:55 Job 'j-2409140c685a4118ac14e40bd9f65b11': queued (progress 0%)
0:01:10 Job 'j-2409140c685a4118ac14e40bd9f65b11': queued (progress 0%)
0:01:30 Job 'j-2409140c685a4118ac14e40bd9f65b11': queued (progress 0%)
0:01:54 Job 'j-2409140c685a4118ac14e40bd9f65b11': queued (progress 0%)
0:02:24 Job 'j-2409140c685a4118ac14e40bd9f65b11': finished (progress 100%)


[PosixPath('output/out.png'), PosixPath('output/job-results.json')]